## Bloomberg BQuant Webinar Series: <br> Integrating RMS into BQuant
This is a companion notebook to the "Integrating RMS into BQuant" webinar.

In [ ]:
from ipywidgets import Tab, HBox, Button, VBox, Dropdown, Layout, HTML, Label, RadioButtons
from bqwidgets import TickerAutoComplete, DataGrid
import bqviz as bqv
import bqplot as bqp
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import bql
bq = bql.Service()

import bqport

In [ ]:
#########################
#
# BQL requests for tab 1 
#
# Created as individual functions so the app creator can quickly execute independently
#
########################

In [ ]:
def BQL_query_all_stocks_with_notes(members):

    query = """
                let(#count = count(group(ID,note_primary_tickers));)
                get(#count)
                for(filter(notes({}), right(NOTE_PRIMARY_TICKERS,6)=='Equity'))
            """.format(members)
    
    response = bq.execute(query)[0].df()
    all_stocks_with_notes = list(response.index)
    
    return all_stocks_with_notes

In [ ]:
def BQL_query_analyst_notes_count(members,time_frame):
    
    query = """
                let(#Count=GROUPSORT(COUNT(GROUP(NOTE_ID(),NOTE_CREATOR())));)
                get(#Count)
                for(filter(NOTES({}),TODAY-NOTE_EVENT_DATE<{}))
            """.format(members,time_frame)

    response = bq.execute(query)[0].df().iloc[:,[-1]]
    group_count_of_analyst_notes = response.head(20)
    
    return group_count_of_analyst_notes

In [ ]:
def BQL_query_stocks_covered_pct_chg(incl_univ):

    query = """
                get(groupsort(DROPNA(PCT_CHG(PX_LAST(dates=range(2020-01-01,today),fill=prev)),remove_id=True)) as #YTD_PCT_CHG)
                for({})
            """.format(incl_univ)

    response = bq.execute(query)[0].df()
    top_10_stocks_covered_pct_chg = response.head(10).reset_index()
    
    return top_10_stocks_covered_pct_chg

In [ ]:
def BQL_query_stocks_not_covered_pct_chg(members,incl_univ):
    
    query = """
                get(groupsort(DROPNA(PCT_CHG(PX_LAST(dates=range(2020-01-01,today),fill=prev)),remove_id=True)) as #YTD_PCT_CHG)
                for(filter({}, NOT(ID_SECURITY_DES IN{})))
            """.format(members,incl_univ)

    response = bq.execute(query)[0].df()
    top_10_stocks_not_covered_pct_chg = response.head(10).reset_index() 
    
    return top_10_stocks_not_covered_pct_chg

In [ ]:
def BQL_query_MoM_time_series(members):

    query = """
                get(groupsort(count(group(ID,[MONTH(note_event_date),NOTE_CREATOR]))) as #NOTE_CREATOR,
                    groupsort(count(group(ID,[MONTH(note_event_date),NOTE_PRIMARY_TICKERS]))) as #NOTE_PRIMARY_TICKERS
                    )
                for(filter(notes({}), right(NOTE_PRIMARY_TICKERS,6)=='Equity'))
            """.format(members)

    responses = bq.execute(query)
       
    response_dict = {}
    for each_response in responses:
        
        df = each_response.df()
        aggregate_by = df.columns[-2:-1].item()
        count = df.columns[-1:].item()
        
        df2 = df.pivot(index = 'MONTH(NOTE_EVENT_DATE)', columns = aggregate_by, values = count)
        df2.index = ['Jan','Feb','March','April','May','June','July','August','Sept','Oct','Nov','Dec']

        df2.loc['total'] = df2.sum()
        df2 = df2.T.nlargest(20,'total').T.iloc[:-1]
        
        response_dict[aggregate_by] = df2
    
    return response_dict

In [ ]:
################################
#
# End of BQL requests for tab 1 
#
################################

In [ ]:
################################
#
# Function which will run all the data for tab1
#
# members and MoM_groups made global as they are used indepently elsewhere
#
################################

In [ ]:
def load_tab1_data():
    
    global members
    global MoM_groups
    
    members = get_univ()

    all_names = BQL_query_all_stocks_with_notes(members) 
    names_covered = BQL_query_stocks_covered_pct_chg(all_names)
    names_not_covered = BQL_query_stocks_not_covered_pct_chg(members,all_names)

    analyst_note_count = BQL_query_analyst_notes_count(members,analyst_time_frame_dd.value[:-1])
    MoM_groups = BQL_query_MoM_time_series(members)
        
    return analyst_note_count,names_covered,names_not_covered,MoM_groups

In [ ]:
################################
#
# Creating the controls for tab 1
#
# get_univ and univ_picker are created so the user can choose whether they look at an equity index or portfolio
# this is then passed into the for() clause of BQL statements
#
################################

In [ ]:
port_list = bqport.list_portfolios()
port_list_dd = [(x['name'],x['id'].split('-')[1].replace(':','-')) for x in port_list]

In [ ]:
universe_dd = Dropdown(description='Universe',options=['Index','Portfolio'])
port_dd = Dropdown(description='Choose Portfolio',options=port_list_dd)
analyst_time_frame_dd = Dropdown(description='Time frame',options=['30d','60d','90d','180d','365d'])

In [ ]:
def get_univ():
    
    if universe_dd.value == 'Index':
        universe = "members('{}')".format(ticker_ac.value)
    if universe_dd.value == 'Portfolio':
        universe = "members('{}',Type=PORT)".format(port_dd.value)
    return universe

In [ ]:
def univ_picker(universe_dd_):
    
    if universe_dd.value == 'Index':
        control.children = [universe_dd,ticker_ac,Go_Button]
    if universe_dd.value == 'Portfolio':
        control.children = [universe_dd,port_dd,Go_Button]

In [ ]:
################################
#
# Create the other parts of the UI
#
################################

In [ ]:
#Default index
universe = 'indu index'

#Controls for first tab
ticker_ac = TickerAutoComplete(description='Type Equity Index:', yellow_keys=['Index'], max_results=5,style={'description_width':'initial'},value=universe)
Go_Button = Button(description='Get data<GO>',button_style='success')
control = HBox([universe_dd,ticker_ac,Go_Button])
groupby_chart_dropdown = Dropdown(description='Group by:',options= ['NOTE_CREATOR','NOTE_PRIMARY_TICKERS'])
hb2 = HBox([analyst_time_frame_dd])

In [ ]:
################################
#
# Create the charts and grids
#
# Create as empty objects so data can be pushed to them after they have been created
#
################################

In [ ]:
#Create other elements for tab1
#Charts
cht_MoM = bqv.BarPlot(title='Monthly time series of notes').set_style()
cht_analyst_count = bqv.BarPlot(title='Number of notes per analyst').set_style()

#column defintions
col_defs = [
            {'headerName': 'Name', 'field': 'ID', 'width': 150, 'headerStyle': {'text-align': 'center'}},
            {'headerName': 'Pct Chg (%)', 'field': '#YTD_PCT_CHG', 'width': 200, 'filter': 'number', 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'center'}}     
            ]

#datagrids
dg1 = DataGrid(data=[], column_defs=col_defs, layout=Layout(width='350px', height='300px'))
dg2 = DataGrid(data=[], column_defs=col_defs, layout=Layout(width='350px', height='300px'))

#Table Tabs
mini_tab = Tab()
mini_tab.set_title(0,'Covered')
mini_tab.set_title(1,'NOT Covered')
mini_tab.children = [dg1,dg2]

In [ ]:
################################
#
# Functions to update all tab1 elements
#
################################

In [ ]:
def update_tab1_elements(cht1_data,grid1,grid2,MoM_groups): # Update for pressing GET NOTES button
    
    cht_analyst_count.push(cht1_data)
    dg1.data = grid1.round(1)
    dg2.data = grid2.round(1)    
    cht_MoM.push(MoM_groups[groupby_chart_dropdown.value])

def update_MoM_groupby(groupby_chart_dropdown_): # Update for changing group by selection on main chart
    
    cht_MoM.push(MoM_groups[groupby_chart_dropdown.value])
    
def update_analyst_count_timeframe(analyst_time_frame_dd_):
    
    hb2.children = [update_view1]
    
    df = BQL_query_analyst_notes_count(members,analyst_time_frame_dd.value[:-1])
    cht_analyst_count.push(df)
    
    hb2.children = [analyst_time_frame_dd]
    

def app_updates(*args):
    
    start_control = list(control.children)
    control.children = list(control.children)[:-1] + [update_view1]
    
    cht1_data,df1,df2,MoM_groups = load_tab1_data()
    update_tab1_elements(cht1_data,df1,df2,MoM_groups)
    
    control.children = list(control.children)[:-1] + [update_view2]
    
    #These are updates for tab 2
    global all_results  
    stock_perf_df,analyst_perf_df,all_results = load_tab2_data()
    update_tab2_elements(stock_perf_df,analyst_perf_df,all_results)
    
    control.children = start_control

In [ ]:
# Something to show the user while app is updating
spinner = HTML('''<i class="fa fa-spinner fa-spin" style="font-size:24px"></i>''')
lbl_update1 = Label('Getting Notes...')
lbl_update2 = Label('Calculating Performance...')
update_view1 = HBox([spinner,lbl_update1])
update_view2 = HBox([spinner,lbl_update2])

In [ ]:
################################
#
# Finally link the interactions on the dropdowns to updating functions with observe()
#
################################

In [ ]:
#Connect the interactions to functions above
groupby_chart_dropdown.observe(update_MoM_groupby)
analyst_time_frame_dd.observe(update_analyst_count_timeframe)
universe_dd.observe(univ_picker)

Go_Button.on_click(app_updates)

In [ ]:
cht1_data,df1,df2,MoM_groups = load_tab1_data()  # Get data for the first time app is run
update_tab1_elements(cht1_data,df1,df2,MoM_groups) # Display data for first time app is run

In [ ]:
#########################
#
# BQL requests for tab 2
#
# Firstly get time series data
# Secondly current information about analyst and note update
#
# 
########################

In [ ]:
#Use dummy data option if no CDE data available
CDE_dummy_data = RadioButtons(
    options=['Use CDE data (fields must be mapped correctly. Click get data<GO> to refresh)', 'Use dummy data'],
    description='Data Source:',
    value='Use dummy data',
    layout={'width': 'max-content'},
    disabled=False
)

In [ ]:
#****CDE fields need to be replaced with firms own fields and populated with data****

analyst_rec = 'UD_BBEQRMS_ANALYST_REC'
research_date = 'UD_BBEQRMS_RESEARCH_DATE'
analyst_name = 'UD_BBEQRMS_ANALYST_NAME'

#***********************

In [ ]:
def BQL_query_get_history(univ):
    
    query = """
                get(DROPNA(PX_LAST(dates=range(-2y,0d))) as #Px,"""+analyst_rec+"""(dates=range(-2y,0d)) as #rec)
                for({})
            """.format(univ)
    
    response = bq.execute(query)
    
    price_time_series = response[0].df()
    rec_time_series = response[1].df()
    
    price_rec_dict = {}
    
    for ticker in list(rec_time_series.index.unique()):
        df1 = price_time_series.loc[[ticker]]
        df2 = rec_time_series.loc[[ticker]]
        df_new = df1.merge(df2,how='outer').fillna(method='bfill').dropna()
        if len(df_new) >1: # Remove any names where we haven't got any recommendations
            price_rec_dict[ticker] = df_new
        
    return price_rec_dict

In [ ]:
def BQL_query_get_current(univ,results_dict):
    
    query = """
                get("""+analyst_name+""",
                """+research_date+""")
                for({})
            """.format(univ)

    df = bq.execute(query)[0].df().reset_index()
    df1 = bq.execute(query)[1].df().reset_index()
    
    df2 = df.merge(df1,how='inner')
    
    df2['new'] = df2.apply(lambda x: add_returns(x['ID'],results_dict),axis=1)
    df2 = df2.dropna()
    df2[research_date] = df2[research_date].astype(str)
    
    return df2

In [ ]:
###############################
#
# End of BQL requests for tab 2
#
###############################

In [ ]:
#########################
#
# Create empty grids and chart for tab 2 like before
#
########################

In [ ]:
#create tab2 grids and chart
col_def1 = [
            {'headerName': 'Name', 'field': 'ID', 'width': 150, 'headerStyle': {'text-align': 'center'}},
            {'headerName': 'Analyst Perf (%)', 'field': 'new', 'width': 200, 'filter': 'number', 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'center'}},
            {'headerName': 'Analyst Name', 'field': analyst_name, 'width': 100, 'filter': 'number', 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'center'}},
            {'headerName': 'Last research date', 'field': research_date, 'width': 150, 'headerStyle': {'text-align': 'center'}}
        ]
dg3 = DataGrid(data=[], column_defs=col_def1, layout=Layout(width='600px', height='300px'))

col_def2 = [
            {'headerName': 'Name', 'field': 'Analyst Name', 'width': 150, 'headerStyle': {'text-align': 'center'}},
            {'headerName': ' Avg Analyst Perf (%)', 'field': 'Avg Analyst Perf (%)', 'width': 200, 'filter': 'number', 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'center'}},
            {'headerName': ' Number of Stocks Covered', 'field': 'Number of Stocks', 'width': 200, 'filter': 'number', 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'center'}},
        ]
dg4 = DataGrid(data=[], column_defs=col_def2, layout=Layout(width='550px', height='300px'))

In [ ]:
################################
#
# Taken from viz gallery, and easily change the axes to fit my data. make sure my data is reshaped first!
#
# x and y marks have been commented out to create a blank object so we can just update the figure when new data is fetched
#
################################

np.random.seed(4)
dates = pd.date_range(end='2019-01-01', periods=120, freq='M')
# dataframe = pd.DataFrame({'Data 1': np.random.randn(120),
#                           'Data 2': np.random.randn(120)*5},
#                          index=dates).cumsum()

dataframe = pd.DataFrame([])

colors=['#1B84ED', '#CF7DFF']

# Create scales
scale_x = bqp.DateScale()
scale_y1 = bqp.LinearScale()
scale_y2 = bqp.LinearScale()

# Create the Lines marks
mark_line1 = bqp.Lines(#x=dataframe['DATE'],
                       #y=dataframe['#Px'],
                       scales={'x': scale_x, 'y': scale_y1},
                       colors=[colors[0]],
                       #labels=[dataframe.columns[0]],
                       display_legend=True)
mark_line2 = bqp.Lines(#x=dataframe['DATE'],
                       #y=dataframe['returns'],
                       scales={'x': scale_x, 'y': scale_y2},
                       colors=[colors[1]],
                       #labels=[dataframe.columns[1]],
                       display_legend=True)

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Dates')
axis_y1 = bqp.Axis(scale=scale_y1,
                   orientation='vertical',
                   label='Price',
                   side='right',
                   tick_style={'fill': colors[0]},
                   label_offset='3em')
axis_y2 = bqp.Axis(scale=scale_y2,
                   orientation='vertical',
                   label='Analyst Performance',
                   grid_lines='none',
                   tick_style={'fill': colors[1]},
                   side='left',
                   label_offset='3em')

# Create Figure
figure = bqp.Figure(marks=[mark_line1, mark_line2],
                    axes=[axis_x, axis_y1, axis_y2],
                    title='Stock Price Vs Analyst Performance',
                    layout={'width':'100%', 'height': '400px'},
                    title_style={'font-size': '22px'},
                    legend_location='top-left',
                    legend_style={'stroke': 'none'},
                    fig_margin={'top': 50, 'bottom': 60,
                                'left': 90, 'right': 90})

# Display the figure
#figure

In [ ]:
#########################
#
# Helper lambda functions that are needed for the pandas manipulation of timeseries data
#
# These are used to calculate returns from buy & sell signals 
#
########################

In [ ]:
def add_returns(t,results_dict):
    #lambda function
    try:
        return (results_dict[t]['returns'][-1:].item()-1)*100
    except:
        return np.nan

def returns(var,rec,px,shifted):
    #lambda function
    if rec == 'Buy' or rec == 'Add' :
        return (px/shifted[var]-1)+1
    if rec == 'Sell':
        return ((px/shifted[var]-1)*-1)+1
    else:
        return 1

In [ ]:
def calc_returns(d):
    results_dict = {}
    for ticker in d:
        df = d[ticker]
        df['returns'] = df.apply(lambda x: returns(x.name,x['#rec'],x['#Px'],df['#Px'].shift()),axis=1).cumprod()
        results_dict[ticker] = df
    
    return results_dict

In [ ]:
def create_analyst_ranking(df):
    
    df2 = pd.pivot_table(df,index=[analyst_name],values=["new"],aggfunc=[np.mean,len]).swaplevel(0,1,axis=1).reset_index()
    df2.columns = df2.columns.droplevel()
    df2.rename(columns={"":"Analyst Name","mean":"Avg Analyst Perf (%)","len":"Number of Stocks"},inplace=True)
    df2 = df2.sort_values(by=['Avg Analyst Perf (%)'],ascending=False)
    
    return df2

In [ ]:
################################
#
# Function which will run all the data for tab2
#
# Using the same members as we used from universe / portfolio dropdown 
#
################################

In [ ]:
def load_tab2_data():
    
    if CDE_dummy_data.value == CDE_dummy_data.options[0]:

        members = get_univ()

        univ = BQL_query_all_stocks_with_notes(members)

        price_rec_dict = BQL_query_get_history(univ)
        results_dict = calc_returns(price_rec_dict)

        df = BQL_query_get_current(univ,results_dict)   
        df2 = create_analyst_ranking(df)
        
    elif CDE_dummy_data.value == CDE_dummy_data.options[1]:
        
        dummy_data1 = {'ID':['V UN Equity', '700 HK Equity', 'IBM UN Equity', 'VOD LN Equity'], 'DATE':['2020-01-10', '2020-01-03', '2020-01-05', '2020-01-17'], 'UD_BBEQRMS_ANALYST_NAME':['John Smith','Jane Doe','John Smith','Ada Lovelace'],'UD_BBEQRMS_RESEARCH_DATE':['2020-01-10', '2020-01-03', '2020-01-05', '2020-01-17'],'new':[5,10,-25,3]}
        df = pd.DataFrame(dummy_data1, index = [0,1,2,3])
        
        dummy_data2 = {'Analyst Name':['John Smith','Jane Doe'],'Avg Analyst Perf (%)':[15,5],'Number of Stocks':[2,1]}
        df2 = pd.DataFrame(dummy_data2, index = [0,1])
        
        today = dt.datetime.today()
        dt1 = today-timedelta(days=60)
        dt2 = today-timedelta(days=100)
        

        results_dict = {'V UN Equity':pd.DataFrame(data = {'DATE':[today,dt1,dt2],'Currency':['USD','USD','USD'],'#Px':[5,7,10],'#rec':['buy','buy','buy'],'returns':[6,9,11]}, index = [0,1,2]),
                      '700 HK Equity':pd.DataFrame(data = {'DATE':[dt1,dt2],'Currency':['USD','USD'],'#Px':[1,3],'#rec':['buy','buy'],'returns':[6,7]}, index = [0,1]),
                      'IBM UN Equity':pd.DataFrame(data = {'DATE':[dt1,dt2],'Currency':['USD','USD'],'#Px':[2,5],'#rec':['buy','buy'],'returns':[6,19]}, index = [0,1]),
                      'VOD LN Equity':pd.DataFrame(data = {'DATE':[dt1,dt2],'Currency':['USD','USD'],'#Px':[15,2],'#rec':['buy','buy'],'returns':[3,11]}, index = [0,1])}
        
    return df, df2, results_dict

In [ ]:
def update_tab2_elements(grid1_data,grid2_data,all_results):
    
    selected_security = grid1_data['ID'][:1].item()
    
    dg3.data = grid1_data.round(1)
    dg4.data = grid2_data.round(1)

    figure.marks[0].y = all_results[selected_security]['#Px']
    figure.marks[1].y = all_results[selected_security]['returns']

In [ ]:
def table_clicked2(*args):
    selected_row = dg3.selected_row_indices
    selected_security = dg3.data.iloc[selected_row]['ID']
    selected_security = selected_security.iloc[0]

    figure.marks[0].x = all_results[selected_security]['DATE']
    figure.marks[1].x = all_results[selected_security]['DATE']
    figure.marks[0].y = all_results[selected_security]['#Px']
    figure.marks[1].y = all_results[selected_security]['returns']
    
    return(selected_security)

In [ ]:
dg3.observe(table_clicked2,'selected_row_indices') # Only one interaction on this tab

In [ ]:
stock_perf_df,analyst_perf_df,all_results = load_tab2_data()
update_tab2_elements(stock_perf_df,analyst_perf_df,all_results)

In [ ]:
#App layout for first tab
app1 = VBox([CDE_dummy_data,control,groupby_chart_dropdown,cht_MoM.show(),hb2,HBox([cht_analyst_count.show(),mini_tab])])
     
#App layout for second tab
app2 = VBox([CDE_dummy_data,control,HBox([dg3,dg4]),figure])

#Set up tabs
tab = Tab()
tab.set_title(0,'NOTE Analysis')
tab.set_title(1,'Analyst Performance')
tab.children = [app1,app2]
tab